<a href="https://colab.research.google.com/github/Parjival/Coding/blob/main/Challenge1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as py
!pip install transformers datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("SKNahin/bengali-transliteration-data")

train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

Training samples: 4004
Validation samples: 1002


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

def preprocess_function(examples):

    banglish_column = train_dataset.column_names[0]
    bangla_column = train_dataset.column_names[1]

    inputs = examples[banglish_column]
    targets = examples[bangla_column]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

In [ ]:
from transformers import MT5ForConditionalGeneration

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
from transformers import Seq2SeqTrainer,Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    predict_with_generate=True
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)



<ipython-input-39-2680af214d65>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
import torch

batch = train_dataset[0:2]

batch = {
    "input_ids": torch.tensor(batch["input_ids"]),
    "labels": torch.tensor(batch["labels"])
}

model_output = model(input_ids=batch["input_ids"], labels=batch["labels"])


print("Model Output Loss:", model_output.loss)

Model Output Loss: tensor(61.8264, grad_fn=<NllLossBackward0>)


In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(val_dataset)


decoded_preds = []
for pred in predictions.predictions:
    try:
        decoded_pred = tokenizer.decode(pred, skip_special_tokens=True)
    except OverflowError:

        decoded_pred = "[UKN]"
    decoded_preds.append(decoded_pred)

decoded_labels = []
for label in predictions.label_ids:
    try:
        decoded_label = tokenizer.decode(label, skip_special_tokens=True)
    except OverflowError:
        decoded_label = "[UKN]"
    decoded_labels.append(decoded_label)


for i in range(5):
    print(f"Input: {val_dataset[i]['Banglish']}")
    print(f"Prediction: {decoded_preds[i]}")
    print(f"Target: {decoded_labels[i]}")
    print("-" * 30)

In [ ]:

model.save_pretrained("./banglish_to_bangla")
tokenizer.save_pretrained("./banglish_to_bangla")